In [1]:
%pip install qdrant-client
%pip install nltk
%pip install -U sentence-transformers
%pip install langdetect
%pip install flask
%pip install flask_restful
%pip install qdrant_client sentence_transformers

In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from langdetect import detect

# Assuming 'text_column' is the column containing text data in your DataFrame
# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'bigBasketProducts.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)



In [35]:
df.head()

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [3]:
df.dtypes

index             int64
product          object
category         object
sub_category     object
brand            object
sale_price      float64
market_price    float64
type             object
rating          float64
description      object
dtype: object

In [4]:
print("Rows before preprocessing:"+str(len(df)))

#Preprocessing the data

#Removing rows that have null values in any one of the column
df.dropna(subset=df.columns, how='any', inplace=True)

print("Rows after preprocessing:"+str(len(df)))

Rows before preprocessing:27555
Rows after preprocessing:18840


In [5]:
print("Rows before preprocessing:"+str(len(df)))

#Removing rows with description < 10 letters
df['description_length'] = df['description'].apply(len)
df = df[df['description_length'] >= 10]   # Drop rows where 'description_length' is less than 10
df.drop(columns=['description_length'], inplace=True) # Drop the temporary 'description_length' column

print("Rows after preprocessing:"+str(len(df)))

Rows before preprocessing:18840
Rows after preprocessing:18839


In [6]:
print("Rows before preprocessing:"+str(len(df)))

#Removing rows that have language in description other than english
# Create a new column 'language' with the detected language for each description
df['language'] = df['description'].apply(lambda x: detect(x) if isinstance(x, str) else '')

# Keep only the rows where the detected language is English ('en')
df = df[df['language'] == 'en']

# Drop the 'language' column as it's no longer needed
df.drop(columns=['language'], inplace=True)


print("Rows after preprocessing:"+str(len(df)))

Rows before preprocessing:18839


Rows after preprocessing:18753


In [7]:
df.dtypes

index             int64
product          object
category         object
sub_category     object
brand            object
sale_price      float64
market_price    float64
type             object
rating          float64
description      object
dtype: object

In [9]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

c:\Users\sandu\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
encoder = SentenceTransformer('multi-qa-MiniLM-L6-dot-v1') # Model to create embeddings

In [11]:
from qdrant_client import QdrantClient

qdrant = QdrantClient(
    url="https://50312433-597c-47df-a1a0-6296d24447d2.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="fMhdYkEkLqxcAvUAAcopmfgnd3puwCU9HbmAlPSm_ZdMs9U-bPOoiA",
)

In [15]:
# Create collection to store vectors
qdrant.recreate_collection(
    collection_name="bigBasketProductsVectors",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [16]:
index=0
df['description'] = df['description'].astype(str)
# Assuming df is your DataFrame

for idx, row in df.iterrows():

    description_vector = encoder.encode(row['description']).tolist()
    if(index%100==0):
      print(str(index))

    index=index+1

    record = models.Record(
        id=idx,
        vector=description_vector,
        payload=row.to_dict()  # Convert the row to a dictionary to use it as payload
    )


    qdrant.upload_records(collection_name="bigBasketProductsVectors", records=[record])


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800


8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700


In [41]:
# Let's now search for something

hits = qdrant.search(
    collection_name="bigBasketProductsVectors",
    query_vector=encoder.encode("potato chips").tolist(),
    limit=20
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'brand': 'Sanjay ', 'category': 'Snacks & Branded Foods', 'description': 'Made with Potato', 'index': 937, 'market_price': 45.0, 'product': 'Chips - Potato, Masala', 'rating': 3.4, 'sale_price': 45.0, 'sub_category': 'Snacks & Namkeen', 'type': 'Chips & Corn Snacks'} score: 0.854321
{'brand': 'A-1 Chips', 'category': 'Snacks & Branded Foods', 'description': 'A-1 Chips Tapioca Chips - Classic Salt 160 g', 'index': 16905, 'market_price': 60.0, 'product': 'Tapioca Chips - Classic Salt', 'rating': 4.1, 'sale_price': 60.0, 'sub_category': 'Snacks & Namkeen', 'type': 'Chips & Corn Snacks'} score: 0.78347826
{'brand': 'A-1 Chips', 'category': 'Snacks & Branded Foods', 'description': 'A-1 Chips Banana Wafers - Ultra Thin, Classic Salt 180 g', 'index': 26357, 'market_price': 100.0, 'product': 'Banana Wafers - Ultra Thin, Classic Salt', 'rating': 4.2, 'sale_price': 100.0, 'sub_category': 'Snacks & Namkeen', 'type': 'Chips & Corn Snacks'} score: 0.77691823
{'brand': 'McCain', 'category': 'Snacks

In [52]:
from transformers import pipeline

# Load the text generation pipeline
text_generator = pipeline('text-generation', model='gpt2')

# Define the query
query = "potato chips"

# Generate a response based on the context of the query
response = text_generator(query, max_length=50, num_return_sequences=1)

# Print the generated response
print(response[0]['generated_text'])


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [51]:
%pip install --upgrade transformers